In [ ]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras.layers import Input
from keras.applications import ResNet50
import PIL
import math
import h5py

In [ ]:
def write_gap(fir, MODEL, image_size, lambda_func=None):
    batch_size = 20
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)
    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))

    gen = ImageDataGenerator()
    train_generator = gen.flow_from_directory(r"D:\data\kaggle\cat_dog\train", image_size, shuffle=False, 
                                              batch_size=batch_size)
    test_generator = gen.flow_from_directory(r"D:\data\kaggle\cat_dog\test", image_size, shuffle=False, 
                                             batch_size=batch_size, class_mode=None)

    # 获取训练数据集的样本数量
    train_samples = math.floor(train_generator.samples/batch_size)
    test_samples = math.floor(test_generator.samples/batch_size)
    print(train_samples)
    print(test_samples)

    train = model.predict(train_generator, steps=train_samples)
    test = model.predict(test_generator, steps=test_samples)
  
 
    # 指定文件路径
    file_path = 'my_gap\\gap_my_'+ fir +'.h5'

    # 创建 HDF5 文件并写入数据
    with h5py.File(file_path, 'w') as h:
        # 写入数据到 HDF5 文件
        h.create_dataset("train", data=train)
        h.create_dataset("test", data=test)
        h.create_dataset("label", data=train_generator.classes)   

In [ ]:
write_gap("ResNet50", ResNet50, (224, 224))

In [ ]:
write_gap("Xception", Xception, (299, 299), xception.preprocess_input)

In [ ]:
write_gap("InceptionV3", InceptionV3, (299, 299), inception_v3.preprocess_input)

In [ ]:
write_gap("VGG16", VGG16, (224, 224))

In [ ]:
write_gap("VGG19", VGG19, (224, 224))